In [7]:
import simpy
import numpy as np

In [41]:
class simulador_servidores:
    mu_llamadas = 0
    mu_db = 0
    mu_db1 = 0
    mu_db2 = 0
    p = 0
    t_esperas = []
    
    def __init__(self, mu_llamadas = 4, mu_db = 0.8, mu_db1 = 0.7, mu_db2 = 1, p = 0.7):
        self.mu_llamadas = mu_llamadas
        self.mu_db = mu_db
        self.p = p
        self.mu_db1 = mu_db1
        self.mu_db2 = mu_db2
        
    def run_alternativa1(self, n, verbose = False):
        self.t_esperas = [[],[]]
        env = simpy.Environment()
        env.process(self._proceso1(env, n, verbose))
        env.run()
        print('Simulación terminada')
        return self.t_esperas
    
    def run_alternativa2(self, n, verbose = False):
        self.t_esperas = []
        env = simpy.Environment()
        env.process(self._proceso2(env, n, verbose))
        env.run()
        print('Simulación terminada')
        return self.t_esperas
    
    def _proceso1(self, env, n, verbose = False):
        i = 0
        while i <= n:
        
            yield env.timeout(np.random.exponential(self.mu_llamadas))
            if verbose:
                print(f'\nLlamada {i} al servicio en el segundo: {env.now}')

            u = np.random.random()
            if u < self.p:
                t_espera = np.random.exponential(self.mu_db1)
                self.t_esperas[0].append(t_espera)
                yield env.timeout(t_espera)
                if verbose:
                    print(f'Atencion por db1 despues de: {t_espera} segundos')

            else:
                t_espera = np.random.exponential(self.mu_db2)
                self.t_esperas[1].append(t_espera)
                yield env.timeout(t_espera)
                if verbose:
                    print(f'Atencion por db2 despues de: {t_espera} segundos')
            i += 1
        
    def _proceso2(self, env, n, verbose = False):
        i = 0
        while i <= n:

            yield env.timeout(np.random.exponential(self.mu_llamadas))
            if verbose:
                print(f'\nLlamada {i} al servicio en el segundo: {env.now}')

            t_espera = np.random.exponential(self.mu_db)
            self.t_esperas.append(t_espera)
            yield env.timeout(t_espera)

            if verbose:
                print(f'Atencion despues de: {t_espera} segundos')

            i += 1

In [42]:
sim = simulador_servidores()
np.array(sim.run_alternativa2(100_000)).sum() / 60 / 24

Simulación terminada


55.904977529001656